In [1]:
config_ini_location = 'config_new.ini'

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']


In [2]:
import openai

# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="""generate a text based on what this person's activity  shows. If it has a negative implication suggest a positive statement  to help and encourage them recover from the negative situation.
  "<?xml version="1.0" encoding="UTF-8"?>
<fitnessActivity>
    <activityDate>2023-10-29</activityDate>
    <startTime>12:00:00</startTime>
    <endTime>12:10:00</endTime>
    <type>Walking</type>
    <duration>60</duration> <!-- Duration in seconds (10 minutes) -->
    <distance unit="meters">300</distance> <!-- Distance covered in meters (300 meters) -->
    <caloriesBurned>20</caloriesBurned>
    <averageHeartRate>80</averageHeartRate>
    <peakHeartRate>90</peakHeartRate>
    <steps>400</steps>
    <location>
        <startLatitude>34.0522</startLatitude>
        <startLongitude>-118.2437</startLongitude>
        <endLatitude>34.0523</endLatitude>
        <endLongitude>-118.2440</endLongitude>
    </location>
    <segments>
        <segment>
            <type>Slow Walk</type>
            <duration>600</duration>
            <distance unit="meters">20</distance>
        </segment>
    </segments>
    <notes>I feel tired and unmotivated.</notes>
</fitnessActivity>

  """,


  max_tokens=50
)



result_text = response.choices[0].text.strip()
print(result_text)


It's okay to feel tired and unmotivated. Why don't you take a break and reward yourself for the walking activity that you've done - you walked 300 meters in 10 minutes and burned 20 calories! That's an impressive feat that's


In [3]:
from audiocraft.models import MusicGen
from audiocraft.models import MultiBandDiffusion

USE_DIFFUSION_DECODER = False
# Using small model, better results would be obtained with `medium` or `large`.
model = MusicGen.get_pretrained('facebook/musicgen-small')
if USE_DIFFUSION_DECODER:
    mbd = MultiBandDiffusion.get_mbd_musicgen()

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1 with CUDA None (you have 2.1.0)
    Python  3.10.4 (you have 3.10.4)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizatio

In [4]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=30
)

In [5]:
import math
import torchaudio
import torch
from audiocraft.utils.notebook import display_audio

def get_bip_bip(bip_duration=0.125, frequency=440,
                duration=0.5, sample_rate=32000, device="cuda"):
    """Generates a series of bip bip at the given frequency."""
    t = torch.arange(
        int(duration * sample_rate), device="cuda", dtype=torch.float) / sample_rate
    wav = torch.cos(2 * math.pi * 440 * t)[None]
    tp = (t % (2 * bip_duration)) / (2 * bip_duration)
    envelope = (tp >= 0.5).float()
    return wav * envelope

In [6]:
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        #'80s pop track with bassy drums and synth',
        #'90s rock song with loud guitars and heavy drums',
        #'Progressive rock drum and bass solo',
        #'Punk Rock song with loud drum and power guitar',
        #'Bluesy guitar instrumental with soulful licks and a driving rhythm section',
        #'Jazz Funk song with slap bass and powerful saxophone',
        
        result_text,
    ],
    progress=True, return_tokens=True
)
display_audio(output[0], sample_rate=32000)
if USE_DIFFUSION_DECODER:
    out_diffusion = mbd.tokens_to_wav(output[1])
    display_audio(out_diffusion, sample_rate=32000)